In [1]:
from counterfactuals.explainers import Fimap
from counterfactuals.visualization import compare
from data import AdultData
from tensorflow import keras
import random
import numpy as np

import warnings
warnings.filterwarnings('ignore')

random.seed(10)
np.random.seed(10)


In [2]:
adult_data = AdultData('data/datasets/adult.csv')
model = keras.models.load_model('models/model_adult')
model_predictions = model.predict(adult_data.X_train)
model_predictions[model_predictions > 0.5] = 1
model_predictions[model_predictions <= 0.5] = 0

fimap = Fimap()
fimap.fit(adult_data.X_train, model_predictions)

test_X = adult_data.X_test.iloc[0]
cf = fimap.generate(test_X)

Epoch 1/5
814/814 [==============================] - 1s 1ms/step - loss: 0.1046 - binary_accuracy: 0.9558
Epoch 2/5
814/814 [==============================] - 1s 1ms/step - loss: 0.0629 - binary_accuracy: 0.9736
Epoch 3/5
814/814 [==============================] - 1s 1ms/step - loss: 0.0539 - binary_accuracy: 0.9773
Epoch 4/5
814/814 [==============================] - 1s 1ms/step - loss: 0.0439 - binary_accuracy: 0.9826
Epoch 5/5
814/814 [==============================] - 1s 1ms/step - loss: 0.0414 - binary_accuracy: 0.9833
Epoch 1/5
814/814 [==============================] - 1s 1ms/step - loss: 0.2166 - binary_accuracy: 0.9262
Epoch 2/5
814/814 [==============================] - 1s 1ms/step - loss: 0.0966 - binary_accuracy: 0.9611
Epoch 3/5
814/814 [==============================] - 1s 1ms/step - loss: 0.0752 - binary_accuracy: 0.9683
Epoch 4/5
814/814 [==============================] - 1s 1ms/step - loss: 0.0623 - binary_accuracy: 0.9744
Epoch 5/5
814/814 [===========================

In [3]:
df = compare(test_X, cf.iloc[0])
df

,X,X',change
relationship,0.000000,1.850094,1.850094
occupation,5.000000,9.256760,4.256760
race,4.000000,3.680502,-0.319498
marital.status,2.000000,1.833457,-0.166543
workclass,3.000000,2.705377,-0.294623
native.country,38.000000,33.442562,-4.557438
sex,1.000000,0.611698,-0.388302
education,15.000000,7.281597,-7.718403
capital.loss,-0.216660,-0.075785,0.140875
age,-0.702455,0.044559,0.747015


In [4]:
from counterfactuals.explainers import ECE

ece = ECE(5, columns=list(adult_data.X_train.columns), bces=[fimap], dist=2, h=8, lambda_=0.2, n_jobs=1)

cf2 = ece.generate(adult_data.X_test.iloc[0])


In [5]:
compare(adult_data.X_test.iloc[0], cf2.iloc[0])

,X,X',change
relationship,0.000000,1.850094,1.850094
occupation,5.000000,9.256760,4.256760
race,4.000000,3.680502,-0.319498
marital.status,2.000000,1.833457,-0.166543
workclass,3.000000,2.705377,-0.294623
native.country,38.000000,33.442562,-4.557438
sex,1.000000,0.611698,-0.388302
education,15.000000,7.281597,-7.718403
capital.loss,-0.216660,-0.075785,0.140875
age,-0.702455,0.044559,0.747015
